# 0 Imports

In [1]:
import requests
import zipfile
import io
import os

# 1 Get data

In [3]:
api_key = os.getenv('API_KEY')

In [2]:
# URL of the CSV file
url = f'https://api.os.uk/downloads/v1/products/OpenUPRN/downloads?area=GB&format=CSV&redirect&key={api_key}'

# Fetch the CSV data from the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Extract the ZIP data
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall('extracted_files')
    print("ZIP file downloaded and extracted successfully.")
else:
    print(f"Failed to fetch data: {response.status_code}")


ZIP file downloaded and extracted successfully.


# 2 Upload data

In [2]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions, GoogleCloudOptions, StandardOptions
import requests
import zipfile
import io
import os
import csv

# URL of the ZIP file
api_key = os.getenv('API_KEY')
url = f'https://api.os.uk/downloads/v1/products/OpenUPRN/downloads?area=GB&format=CSV&redirect&key={api_key}'

# Fetch the ZIP data from the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Extract the ZIP data
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall('extracted_files')
    print("ZIP file downloaded and extracted successfully.")
else:
    print(f"Failed to fetch data: {response.status_code}")

# Path to the extracted CSV file
csv_file_path = 'extracted_files/osopenuprn_202406.csv'  # Replace with the actual CSV file name

# Define the pipeline options
options = PipelineOptions()
google_cloud_options = options.view_as(GoogleCloudOptions)
google_cloud_options.project = 'os-data-433106'  # Replace with your GCP project ID
google_cloud_options.job_name = 'csv-to-bigquery'
google_cloud_options.staging_location = 'gs://os-data/staging'  # Replace with your GCS bucket
google_cloud_options.temp_location = 'gs://os-data/temp'  # Replace with your GCS bucket
options.view_as(StandardOptions).runner = 'DataflowRunner'

# Define the BigQuery schema
table_schema = 'UPRN:INT64,X_COORDINATE:FLOAT64,Y_COORDINATE:FLOAT64,LATITUDE:FLOAT64,LONGITUDE:FLOAT64'  # Replace with your actual schema

# Define the pipeline
with beam.Pipeline(options=options) as p:
    (p
     | 'Read CSV' >> beam.io.ReadFromText(csv_file_path, skip_header_lines=1)
     | 'Parse CSV' >> beam.Map(lambda line: next(csv.reader([line])))
     | 'To Dict' >> beam.Map(lambda fields: {'column1': fields[0], 'column2': fields[1], 'column3': fields[2]})  # Adjust as needed
     | 'Write to BigQuery' >> beam.io.WriteToBigQuery(
            'os-data-433106.os.os-open-uprn',  # Replace with your dataset and table ID
            schema=table_schema,
            write_disposition=beam.io.BigQueryDisposition.WRITE_TRUNCATE,
            create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED)
    )

Failed to fetch data: 401


usage: ipykernel_launcher.py [-h] [--dataflow_endpoint DATAFLOW_ENDPOINT]
                             [--project PROJECT] [--job_name JOB_NAME]
                             [--staging_location STAGING_LOCATION]
                             [--temp_location TEMP_LOCATION] [--region REGION]
                             [--service_account_email SERVICE_ACCOUNT_EMAIL]
                             [--no_auth]
                             [--template_location TEMPLATE_LOCATION]
                             [--label LABELS] [--update]
                             [--transform_name_mapping TRANSFORM_NAME_MAPPING]
                             [--enable_streaming_engine]
                             [--dataflow_kms_key DATAFLOW_KMS_KEY]
                             [--create_from_snapshot CREATE_FROM_SNAPSHOT]
                             [--flexrs_goal {COST_OPTIMIZED,SPEED_OPTIMIZED}]
                             [--dataflow_service_option DATAFLOW_SERVICE_OPTIONS]
                           

AttributeError: 'tuple' object has no attribute 'tb_frame'